# A Visual Notebook to Using BERT for the First Time

In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the text. The text is a list of tweets from celebrities. And we will classify each sentence with the celebrity that tweeted it.

## Models: Sentence Celebrity Classification
Our goal is to create a model that takes a sentence (just like the ones in our dataset) and predict the celebrity. We can think of it as looking like this:

Under the hood, the model is actually made up of two model.

* DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
* The next model, a basic Logistic Regression model from scikit learn will take in the result of DistilBERT’s processing, and classify the sentence with the celebrity.

The data we pass **between** the two models is a vector of size 768. We can think of this of vector as an embedding for the sentence that we can use for classification.

## Dataset
The dataset we will use in this example is [here](https://github.com/estorrs/twitter-celebrity-tweet-sentiment/blob/master/results/celebrity_tweets_results.csv), which contains tweets, each labeled with the celebrity and whether it was a positive or negative tweet but now we just care about the celebrity:

## Installing the transformers library
Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [ ]:
!pip install -q transformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [ ]:
df = pd.read_csv('', header=None)
df = df.sample(frac=1).reset_index(drop=True)

Let's quickly look at which celebrities are in the dataset.


In [ ]:
df[0]

In [ ]:
df[1]

In [ ]:
df[0].unique()

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
tokenized = df[1].apply(lambda x: tokenizer.encode(str(x), add_special_tokens=True))

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding
After tokenization, `tokenized` is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [ ]:
np.array(padded).shape

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
example = df[1][53]
example

In [ ]:
example.split()

In [ ]:
tokenized.values[53]

In [ ]:
padded[53]

In [ ]:
attention_mask[53]

## Model #1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model!

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

For time sake, we will only use 300 tweets to get features from.

In [ ]:
input_ids = torch.tensor(padded)[:300] 
attention_mask = torch.tensor(attention_mask)[:300]

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called `[CLS]` (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

We'll save those in the `features` variable, as they'll serve as the features to our logitics regression model.

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [ ]:
labels = df[0][:300]

In [ ]:
labels.shape

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 300 sentences from the dataset).

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, shuffle=False)

In [ ]:
train_labels

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)


## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [ ]:
lr_clf.score(test_features, test_labels)



## Improve Scores

And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.

## Quiz the Audience

Let's play a game and test our `model`, and audience, with which celebrity said which quote.



Here's are the celebrities in the dataset to choose from.

In [ ]:
df[0].unique()

In [ ]:
random_number = 13 # Put your own! Pick from 0 - 74
df[1][df.shape[0] - test_features.shape[0] + random_number]

In [ ]:
lr_clf.predict(test_features[random_number, np.newaxis])

In [ ]:
test_labels.iloc[random_number]

## The End: Additional Resources

Here are some extra resources to look at to get a deeper insight for BERT and everything we've talked about:



*   Understanding BERT: https://www.analyticsvidhya.com/blog/2019/09/demystifying-bert-groundbreaking-nlp-framework/

*   More on Word2Vec: https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa

*   Hugging Face Library Documentation: https://huggingface.co/

*   More info for Transformers: https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04

*   Attention Mechanism Explained: https://towardsdatascience.com/intuitive-understanding-of-attention-mechanism-in-deep-learning-6c9482aecf4f

*   Using ELI5 Library for Keras BERT: https://colab.research.google.com/drive/1YxcceZxsNlvK35pRURgbwvkgejXwFxUt#scrollTo=B4R1r12rgNlI&forceEdit=true&sandboxMode=true

